In [1]:
# Import necessary libraries
import pandas as pd
from sklearn.linear_model import LinearRegression
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.model_selection import train_test_split

In [2]:
# Set max columns to display
pd.set_option('display.max_columns', None)

In [3]:
# df_All = df_All[df_All['Date'] > '2022-06-15 00:00:00']

In [4]:
df_All_1 = pd.read_csv(r'C:\Users\saust\OneDrive\Desktop\GitRepo\Project-OptiC4\1 Preprocess\Merge Data\contData_all_Avg_1o2.csv')
df_All_2 = pd.read_csv(r'C:\Users\saust\OneDrive\Desktop\GitRepo\Project-OptiC4\1 Preprocess\Merge Data\contData_all_Avg_2o2.csv')
# Concatenate (union) the dataframes
df_All = pd.concat([df_All_1, df_All_2], ignore_index=True)

print(df_All.head())

   425_pct_Al  M_Value  C4_pct_Eth  C4_pct_H2O  HydWtr_pct_Ammonia   DI55152  \
0    6.307630  3.50893    2.508420    21.96340            0.908765  0.925255   
1    6.308335  3.50863    2.511135    21.96865            0.907643  0.924281   
2    6.309390  3.50818    2.515210    21.97655            0.905959  0.923773   
3    6.310090  3.50788    2.517925    21.98185            0.904837  0.924103   
4    6.310790  3.50758    2.520640    21.98715            0.903715  0.924258   

    FC55003       FC55552      FC55569  FFC55553  FFC55555    LC55555  \
0  4919.290  41564.100000  6918.110000  0.996975  0.751501  59.968800   
1  5039.445  41558.250000  6919.960000  0.998847  0.750588  59.932300   
2  5268.620  41524.066667  6916.486667  0.998648  0.750505  60.002433   
3  5349.320  41500.275000  6918.470000  1.000349  0.750325  60.015925   
4  5563.165  41466.625000  6915.790000  1.000862  0.750021  60.020500   

    PI55004   TC55552     TI55021    Butanol  
0  1.306310  177.1880  212.861000

In [5]:
# # List of columns to exclude to run XGboost feature selection
# exclude_columns = ['Octanol', 'Hexanol', 'Ethanol', 'Decanol',
       
#        'TI52014', 'TI55013', 'TI55014', 'TI55015', 'TI55016', 'TI55017', 'TI55021', 'TI55023',
#        'TC52015', 'FC52018', 'II52554', 'TI40050', 'VI52558B'
#                    ]

# # Create a new DataFrame without the excluded columnsd
# df_All = df_All.drop(columns=exclude_columns)

In [6]:
# selected_columns = ['425 %Al', 'Butanol', 'DI55102', 'DI55152', 'DI55580', 'FC42428',
#        'FC55003', 'FC55552', 'FC55569', 'FC55576', 'FFC55553', 'FFC55555',
#        'LC52572', 'LC55553', 'LC55555', 'LC55557', 'LC55568', 'LC90366',
#        'LC90368', 'PI55004', 'PI55020', 'TC55552', 'TC55553', 'TC55555'
#                    ]

# existing_columns = [col for col in selected_columns if col in df_All.columns]
# df_All = df_All[existing_columns]

In [7]:
df_All.columns

Index(['425_pct_Al', 'M_Value', 'C4_pct_Eth', 'C4_pct_H2O',
       'HydWtr_pct_Ammonia', 'DI55152', 'FC55003', 'FC55552', 'FC55569',
       'FFC55553', 'FFC55555', 'LC55555', 'PI55004', 'TC55552', 'TI55021',
       'Butanol'],
      dtype='object')

In [8]:
# Splitting into train and test
X = df_All.drop('Butanol', axis=1)  # Assuming 'target' is your target column
y = df_All['Butanol']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
# Create a linear regression model
linreg = LinearRegression()

# Create a SequentialFeatureSelector object
sfs = SFS(linreg, 
          k_features='best', 
          forward=True, 
          floating=True, 
          scoring='r2',
          cv=5)

# Fit the model
sfs = sfs.fit(X_train, y_train)

# Get the selected feature names
selected_features = X_train.columns[list(sfs.k_feature_idx_)]

# Build the final model using selected features
final_model = linreg.fit(X_train[selected_features], y_train)

# Optionally, evaluate the model on the test set
y_pred = final_model.predict(X_test[selected_features])

In [10]:
# Get the selected feature names
selected_features = X_train.columns[list(sfs.k_feature_idx_)]
print("Selected Features:")
print(selected_features)

# Display the coefficients of the final model
print("\nModel Coefficients:")
for i, feature in enumerate(selected_features):
    print(f"{feature}: {final_model.coef_[i]}")

# Display the R-squared value for the training set
r_squared_train = final_model.score(X_train[selected_features], y_train)
print(f"\nR-squared on Training Set: {r_squared_train}")

# Optionally, if you evaluated the model on a test set
r_squared_test = final_model.score(X_test[selected_features], y_test)
print(f"R-squared on Test Set: {r_squared_test}")


Selected Features:
Index(['425_pct_Al', 'M_Value', 'C4_pct_Eth', 'C4_pct_H2O', 'DI55152',
       'FC55003', 'FC55552', 'FC55569', 'FFC55553', 'FFC55555', 'LC55555',
       'PI55004', 'TC55552', 'TI55021'],
      dtype='object')

Model Coefficients:
425_pct_Al: -0.29413171840183144
M_Value: -0.5775288879561625
C4_pct_Eth: -0.6327316637626352
C4_pct_H2O: 0.3120305396844868
DI55152: 10.676830903263683
FC55003: -0.0011093770818439124
FC55552: 0.0004528184124492764
FC55569: -0.003776945933543135
FFC55553: 30.693665727497528
FFC55555: -50.94809180374684
LC55555: 0.04595258426793167
PI55004: 0.2887185699226437
TC55552: 0.04666535199570244
TI55021: -0.09995652449977958

R-squared on Training Set: 0.12374620762809707
R-squared on Test Set: 0.12121759141147848
